In [ ]:
!pip3 install --upgrade --user google-genai
!pip3 install -U -q httpx -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [ ]:
MODEL = "gemini-2.5-pro"
LOCATION = "us-central1"
PROJECT_ID = "veo-27350"

ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

In [ ]:
import base64
import httpx
import requests
from IPython.display import Image

In [ ]:
from google import genai
from google.genai import types
client = genai.Client(
  vertexai=True, project="YOUR_PROJECT_ID", location="global",
)
# If your image is stored in Google Cloud Storage, you can use the from_uri class method to create a Part object.
IMAGE_URI = "gs://generativeai-downloads/images/scones.jpg"
model = "gemini-2.5-pro"
response = client.models.generate_content(
  model=model,
  contents=[
    "What is shown in this image?",
    types.Part.from_uri(
      file_uri=IMAGE_URI,
      mime_type="image/png",
    ),
  ],
)
print(response.text, end="")

ClientError: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Permission denied on resource project YOUR_PROJECT_ID.', 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'CONSUMER_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'aiplatform.googleapis.com', 'consumer': 'projects/YOUR_PROJECT_ID', 'containerInfo': 'YOUR_PROJECT_ID'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'Permission denied on resource project YOUR_PROJECT_ID.'}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developers console', 'url': 'https://console.developers.google.com'}]}]}}

In [ ]:
# SDK-based OCR processing for Gemini 2.5 Pro via Vertex AI
import os
import csv
import time
import re
import traceback
from google import genai
from google.genai import types

# Create a client for Gemini API in Vertex AI
# PROJECT_ID and LOCATION should already be defined in previous cells.
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

# Generation configuration
GENERATION_CONFIG = {
    "temperature": 0.2,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

MODEL_NAME = "gemini-2.5-pro-preview-03-25"

# System prompts
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

def extract_folder_info(folder_path):
    if not os.path.isdir(folder_path):
        return None, None
    full_name = os.path.basename(folder_path)
    return full_name, full_name

def sanitize_filename(filename):
    sanitized = filename.strip()
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    sanitized = sanitized.replace('/', '_')
    return sanitized

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def process_folder_sdk(folder_path):
    if not os.path.isdir(folder_path):
        print(f"🛑 Error: Target folder not found at '{folder_path}'")
        return

    print(f"\nStarting SDK-based OCR for folder: '{os.path.basename(folder_path)}'")

    sujet_copie, nom_complet_sous_dossier = extract_folder_info(folder_path)
    if not sujet_copie:
        print(f"🛑 Error: Couldn't extract info from folder path '{folder_path}'")
        return

    parent_dir = os.path.dirname(folder_path)
    sanitized_name = sanitize_filename(nom_complet_sous_dossier)
    output_csv_path = os.path.join(parent_dir, f"{sanitized_name}_sdk.csv")
    print(f"  Output CSV will be: '{output_csv_path}'")

    try:
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([nom_complet_sous_dossier])

            try:
                all_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.png')]
                sorted_chunks = sorted(all_files, key=natural_sort_key)
            except OSError as e:
                print(f"  🛑 Error listing files: {e}")
                sorted_chunks = []

            if not sorted_chunks:
                print("  No PNG files found.")
                return

            print(f"  Found {len(sorted_chunks)} chunks to process.")

            ocr_result_n_minus_2 = ""
            ocr_result_n_minus_1 = ""

            for i, chunk_filename in enumerate(sorted_chunks):
                chunk_path = os.path.join(folder_path, chunk_filename)
                current_ocr_result = ""

                try:
                    if i == 0:
                        system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                        contents = []
                    else:
                        system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                        context_text = "\n--- Texte Précédent ---\n"
                        if ocr_result_n_minus_2:
                            context_text += ocr_result_n_minus_2 + "\n"
                        if ocr_result_n_minus_1:
                            context_text += ocr_result_n_minus_1
                        context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                        contents = [types.Part.from_text(context_text)]

                    with open(chunk_path, 'rb') as img_file:
                        image_bytes = img_file.read()
                    contents.append(types.Part.from_bytes(image_bytes, mime_type='image/png'))

                    response = client.models.generate_content(
                        model=MODEL_NAME,
                        contents=contents,
                        system_instruction=system_instruction,
                        generation_config=GENERATION_CONFIG,
                        request_options={'timeout': 600},
                    )

                    if response and hasattr(response, 'text') and response.text:
                        current_ocr_result = response.text.strip()
                    else:
                        current_ocr_result = "ERROR: Empty response"

                    writer.writerow([current_ocr_result])

                    ocr_result_n_minus_2 = ocr_result_n_minus_1
                    ocr_result_n_minus_1 = current_ocr_result

                except Exception as e:
                    print(f"🛑 Error processing {chunk_filename}: {e}")
                    traceback.print_exc()
                    writer.writerow([f"ERROR: {type(e).__name__}"])

                finally:
                    time.sleep(1.5)

    except Exception as e:
        print(f"🛑 Unexpected error: {e}")
        traceback.print_exc()

    print(f"\nCompleted processing (SDK) for: {os.path.basename(folder_path)}")
    print(f"CSV saved at: {output_csv_path}")


In [ ]:
# SDK-based OCR processing for Gemini 2.5 Pro via Vertex AI
import os
import csv
import time
import re
import traceback
from google import genai
from google.genai import types

# Create a client for Gemini API in Vertex AI
# PROJECT_ID and LOCATION should already be defined in previous cells.
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

# Generation configuration
GENERATION_CONFIG = {
    "temperature": 0.2,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

MODEL_NAME = "gemini-2.5-pro-preview-03-25"

# System prompts
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

def extract_folder_info(folder_path):
    if not os.path.isdir(folder_path):
        return None, None
    full_name = os.path.basename(folder_path)
    return full_name, full_name

def sanitize_filename(filename):
    sanitized = filename.strip()
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    sanitized = sanitized.replace('/', '_')
    return sanitized

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def process_folder_sdk(folder_path):
    if not os.path.isdir(folder_path):
        print(f"🛑 Error: Target folder not found at '{folder_path}'")
        return

    print(f"\nStarting SDK-based OCR for folder: '{os.path.basename(folder_path)}'")

    sujet_copie, nom_complet_sous_dossier = extract_folder_info(folder_path)
    if not sujet_copie:
        print(f"🛑 Error: Couldn't extract info from folder path '{folder_path}'")
        return

    parent_dir = os.path.dirname(folder_path)
    sanitized_name = sanitize_filename(nom_complet_sous_dossier)
    output_csv_path = os.path.join(parent_dir, f"{sanitized_name}_sdk.csv")
    print(f"  Output CSV will be: '{output_csv_path}'")

    try:
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([nom_complet_sous_dossier])

            try:
                all_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.png')]
                sorted_chunks = sorted(all_files, key=natural_sort_key)
            except OSError as e:
                print(f"  🛑 Error listing files: {e}")
                sorted_chunks = []

            if not sorted_chunks:
                print("  No PNG files found.")
                return

            print(f"  Found {len(sorted_chunks)} chunks to process.")

            ocr_result_n_minus_2 = ""
            ocr_result_n_minus_1 = ""

            for i, chunk_filename in enumerate(sorted_chunks):
                chunk_path = os.path.join(folder_path, chunk_filename)
                current_ocr_result = ""

                try:
                    if i == 0:
                        system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                        contents = []
                    else:
                        system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                        context_text = "\n--- Texte Précédent ---\n"
                        if ocr_result_n_minus_2:
                            context_text += ocr_result_n_minus_2 + "\n"
                        if ocr_result_n_minus_1:
                            context_text += ocr_result_n_minus_1
                        context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                        contents = [types.Part.from_text(context_text)]

                    with open(chunk_path, 'rb') as img_file:
                        image_bytes = img_file.read()
                    contents.append(types.Part.from_bytes(image_bytes, mime_type='image/png'))

                    response = client.models.generate_content(
                        model=MODEL_NAME,
                        contents=contents,
                        system_instruction=system_instruction,
                        generation_config=GENERATION_CONFIG,
                        request_options={'timeout': 600},
                    )

                    if response and hasattr(response, 'text') and response.text:
                        current_ocr_result = response.text.strip()
                    else:
                        current_ocr_result = "ERROR: Empty response"

                    writer.writerow([current_ocr_result])

                    ocr_result_n_minus_2 = ocr_result_n_minus_1
                    ocr_result_n_minus_1 = current_ocr_result

                except Exception as e:
                    print(f"🛑 Error processing {chunk_filename}: {e}")
                    traceback.print_exc()
                    writer.writerow([f"ERROR: {type(e).__name__}"])

                finally:
                    time.sleep(1.5)

    except Exception as e:
        print(f"🛑 Unexpected error: {e}")
        traceback.print_exc()

    print(f"\nCompleted processing (SDK) for: {os.path.basename(folder_path)}")
    print(f"CSV saved at: {output_csv_path}")


In [ ]:
/content/drive/MyDrive/folder_partitioned_merged

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/9/95/Leibniz_Manuscript_of_integral_and_differential_notation.png"
image_media_type = "image/png"
image_b64 = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

response = requests.get(image_url)
image = Image(response.content, width=300, height=200)

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_b64,
                    },
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ],
    model=MODEL,
)
print(message.model_dump_json(indent=2))

{
  "id": "msg_vrtx_018rENYF44iN943fJLy6Sb3a",
  "content": [
    {
      "text": "This image appears to be a historical mathematical manuscript or notebook page. It's filled with handwritten mathematical equations, symbols, and notations in what looks like Latin or an older form of scholarly writing. The page is yellowed with age, suggesting it's quite old.\n\nThe content includes various mathematical expressions, including square roots, integrals, and other complex mathematical notations. There are also some small geometric sketches or diagrams in the corners of the page.\n\nThe handwriting is in a flowing, cursive style typical of older manuscripts. The text is dense and covers most of the page, with different sizes of writing and some words or phrases crossed out or corrected.\n\nThis document appears to be a working draft or notes from a mathematician, possibly exploring complex mathematical concepts or working through proofs. It's a fascinating glimpse into the thought process an

In [ ]:
# --- OCR processing using Gemini 2.5 Pro via GCP SDK (Vertex AI) ---
# This cell adapts the previous OCR pipeline to use the Google Gen AI SDK with Vertex AI credentials.
# It uses the existing PROJECT_ID, LOCATION and MODEL variables defined earlier.

from google import genai
from google.genai import types

import os
import csv
import time
import re
from tqdm.notebook import tqdm
import traceback

# Initialize Vertex AI client
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

# Generation configuration for the model
GENERATION_CONFIG_SDK = {
    "temperature": 0.2,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
}

# Use the same model name as defined earlier
MODEL_NAME_SDK = MODEL  # e.g., "gemini-2.5-pro"

# System instructions (copied from previous code)
SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

def extract_folder_info(folder_path):
    """Extracts the full folder name while preserving all characters."""
    if not os.path.isdir(folder_path):
        return None, None
    full_name = os.path.basename(folder_path)
    return full_name, full_name

def sanitize_filename(filename):
    """Removes or replaces characters invalid for filenames."""
    sanitized = filename.strip()
    sanitized = re.sub(r'[\\\\/*?:"<>|]', '_', sanitized)
    sanitized = sanitized.replace('/', '_')
    return sanitized

def natural_sort_key(s):
    """Key for sorting strings containing numbers naturally."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def process_folder_sdk(folder_path):
    """
    Processes all PNG images in the specified folder using Vertex AI Gemini 2.5 Pro.
    Generates OCR for each image and writes the results into a CSV file.
    """
    if not os.path.isdir(folder_path):
        print(f"🛑 Error: Target folder not found at '{folder_path}'")
        return

    print(f"\nStarting OCR process (SDK) for folder: '{os.path.basename(folder_path)}'")

    # Extract subject and folder name
    sujet_copie, nom_complet_sous_dossier = extract_folder_info(folder_path)
    if not sujet_copie:
        print(f"🛑 Error: Couldn't extract info from folder path '{folder_path}'")
        return

    print(f"  Subject detected: '{sujet_copie}'")

    # Output CSV path
    parent_dir = os.path.dirname(folder_path)
    sanitized_full_name = sanitize_filename(nom_complet_sous_dossier)
    output_csv_path = os.path.join(parent_dir, f"{sanitized_full_name}_sdk.csv")
    print(f"  Output CSV will be: '{output_csv_path}'")

    try:
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([nom_complet_sous_dossier])

            # List and sort PNG files
            try:
                all_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".png")]
                sorted_chunks = sorted(all_files, key=natural_sort_key)
            except OSError as e:
                print(f"  🛑 Error listing files: {e}")
                sorted_chunks = []

            if not sorted_chunks:
                print("  No PNG files found.")
                return

            print(f"  Found {len(sorted_chunks)} chunks to process (SDK).")
            ocr_result_n_minus_1 = ""
            ocr_result_n_minus_2 = ""

            for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks (SDK)")):
                chunk_path = os.path.join(folder_path, chunk_filename)
                current_ocr_result = ""

                try:
                    # Choose system instruction and build context
                    if i == 0:
                        system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                        prompt_parts = []
                    else:
                        system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                        context_text = "\n--- Texte Précédent ---\n"
                        if ocr_result_n_minus_2:
                            context_text += ocr_result_n_minus_2 + "\n"
                        if ocr_result_n_minus_1:
                            context_text += ocr_result_n_minus_1
                        context_text += "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                        prompt_parts = [context_text]

                    # Read image bytes
                    with open(chunk_path, 'rb') as f:
                        image_bytes = f.read()

                    # Append image as Part
                    image_part = types.Part.from_bytes(data=image_bytes, mime_type="image/png")
                    prompt_parts.append(image_part)

                    # Build generation config including system instruction
                    gen_config = types.GenerateContentConfig(
                        system_instruction=system_instruction,
                        temperature=GENERATION_CONFIG_SDK["temperature"],
                        top_p=GENERATION_CONFIG_SDK["top_p"],
                        top_k=GENERATION_CONFIG_SDK["top_k"],
                        max_output_tokens=GENERATION_CONFIG_SDK["max_output_tokens"],
                    )

                    # API call via SDK
                    response = client.models.generate_content(
                        model=MODEL_NAME_SDK,
                        contents=prompt_parts,
                        config=gen_config,
                    )

                    if response and hasattr(response, "text") and response.text:
                        current_ocr_result = response.text.strip()
                    else:
                        current_ocr_result = "ERROR: Empty response"

                    writer.writerow([current_ocr_result])

                    # Update history for context
                    ocr_result_n_minus_2 = ocr_result_n_minus_1
                    ocr_result_n_minus_1 = current_ocr_result

                except Exception as e_api:
                    print(f"🛑 Error processing {chunk_filename}:")
                    traceback.print_exc()
                    writer.writerow([f"ERROR: {type(e_api).__name__}"])

                finally:
                    # small delay to respect rate limits
                    time.sleep(1.5)

    except IOError as e:
        print(f"🛑 CSV Error: {e}")
    except Exception as e_main:
        print(f"🛑 Unexpected error:")
        traceback.print_exc()

print(f"\nCompleted processing (SDK) for: {os.path.basename(folder_path)}#")pri#nt(f"\nCompleted processing (SDK {os.path.basename(folder_path)}")
#    print(f"CSV saved at: {output_csv_pattest
print(f"CSV saved at: {output_csv_path}"
)
print(f"\nCompleted processing (SDK) for: {os.path.basename(folder_path)}")
print(f"CSV saved at: {output_csv_path}"print(f"\nCompleted processing (SDK) for: {os.path.basename(folder_path)}")
print(f"CSV
 saved at: {output_csv_path}"

SyntaxError: invalid syntax (ipython-input-2852869130.py, line 185)

In [ ]:
# --- OCR processing using Gemini 2.5 Pro via GCP SDK (Vertex AI) ---
# Fusion: boucle PNG + appel SDK Vertex (sans API key Ai Studio)

from google import genai
from google.genai import types

import os
import csv
import time
import re
from tqdm.notebook import tqdm
import traceback

# Client Vertex AI (repose sur ton auth GCP active dans ce Colab)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

# Config « gabarit » (sans system_instruction ici)
BASE_GENERATION_CONFIG = dict(
    temperature=0.2,
    top_p=0.95,
    top_k=64,
    max_output_tokens=8192,
)

MODEL_NAME_SDK = MODEL  # ex. "gemini-2.5-pro"

SYSTEM_INSTRUCTION_WITH_CONTEXT = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}".

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).

Aide Contextuelle (Texte Précédent) : Pour t'aider à déchiffrer l'écriture, le texte qui précède *immédiatement* celui de l'image actuelle est fourni ci-dessous (séparé par '--- Texte Précédent ---'). Ce texte a été obtenu par OCR des sections précédentes de la dissertation. Le texte de l'image actuelle commence là où le texte précédent s'arrête (il peut y avoir un léger chevauchement de lignes dû au découpage des images).

**Priorité :** Ta tâche principale reste de transcrire *exactement* ce que tu vois sur l'image actuelle. Utilise le texte précédent **uniquement** comme une aide *secondaire* pour résoudre des ambiguïtés ou déchiffrer des mots très difficiles. **Ne laisse PAS le texte précédent remplacer ou modifier ce qui est clairement visible sur l'image actuelle.** En cas de conflit entre le texte précédent et l'image, **la transcription fidèle de l'image PRÉVAUT.**
"""

SYSTEM_INSTRUCTION_FIRST_CHUNK = """Tâche Principale : Recopie aussi fidèlement que possible le texte manuscrit de l'image fournie. Répond **exclusivement** avec le texte transcrit, sans aucune introduction, commentaire, ou note.

Contexte de la Dissertation : Ce texte est un extrait d'une dissertation de philosophie portant sur le sujet "{sujet_copie}". Ceci est le tout début de la dissertation.

Mise en forme :
- Si des mots ou groupes de mots sont soulignés dans l'image, mets-les en italique en markdown (*texte souligné*).
- Si un mot est barré, encadre-le de tildes en markdown (~mot barré~).
"""

def extract_folder_info(folder_path):
    if not os.path.isdir(folder_path):
        return None, None
    full_name = os.path.basename(folder_path)
    return full_name, full_name

def sanitize_filename(filename):
    sanitized = filename.strip()
    sanitized = re.sub(r'[\\/*?:"<>|]', '_', sanitized)
    sanitized = sanitized.replace('/', '_')
    return sanitized

def natural_sort_key(s):
    return [int(t) if t.isdigit() else t.lower() for t in re.split('([0-9]+)', s)]

def process_folder_sdk(folder_path):
    if not os.path.isdir(folder_path):
        print(f"\U0001F198 Error: Target folder not found at '{folder_path}'")
        return

    print(f"\nStarting OCR process (SDK) for folder: '{os.path.basename(folder_path)}'")

    sujet_copie, nom_complet_sous_dossier = extract_folder_info(folder_path)
    if not sujet_copie:
        print(f"\U0001F198 Error: Couldn't extract info from folder path '{folder_path}'")
        return

    print(f"  Subject detected: '{sujet_copie}'")

    parent_dir = os.path.dirname(folder_path)
    sanitized_full_name = sanitize_filename(nom_complet_sous_dossier)
    output_csv_path = os.path.join(parent_dir, f"{sanitized_full_name}_sdk.csv")
    print(f"  Output CSV will be: '{output_csv_path}'")

    try:
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([nom_complet_sous_dossier])

            try:
                all_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.png')]
                sorted_chunks = sorted(all_files, key=natural_sort_key)
            except OSError as e:
                print(f"  \U0001F198 Error listing files: {e}")
                sorted_chunks = []

            if not sorted_chunks:
                print("  No PNG files found.")
                return

            print(f"  Found {len(sorted_chunks)} chunks to process (SDK).")

            ocr_result_n_minus_1 = ''
            ocr_result_n_minus_2 = ''

            for i, chunk_filename in enumerate(tqdm(sorted_chunks, desc="Processing Chunks (SDK)")):
                chunk_path = os.path.join(folder_path, chunk_filename)
                current_ocr_result = ''

                try:
                    # Instruction système et éventuel contexte
                    if i == 0:
                        system_instruction = SYSTEM_INSTRUCTION_FIRST_CHUNK.format(sujet_copie=sujet_copie)
                        contents = []
                    else:
                        system_instruction = SYSTEM_INSTRUCTION_WITH_CONTEXT.format(sujet_copie=sujet_copie)
                        context_text = ""
                        if ocr_result_n_minus_2:
                            context_text += ocr_result_n_minus_2 + "\n"
                        if ocr_result_n_minus_1:
                            context_text += ocr_result_n_minus_1
                        context_text = (
                            "\n--- Texte Précédent ---\n" + context_text +
                            "\n--- Fin du Texte Précédent ---\n\n--- Image Actuelle à Transcrire ---"
                        )
                        contents = [types.Part.from_text(text=context_text)]  # ⚠️ argument nommé

                    # Image -> Part (⚠️ arguments nommés)
                    with open(chunk_path, 'rb') as f:
                        image_bytes = f.read()
                    image_part = types.Part.from_bytes(data=image_bytes, mime_type='image/png')
                    contents.append(image_part)

                    # Build config à la volée avec system_instruction
                    req_config = types.GenerateContentConfig(
                        **BASE_GENERATION_CONFIG,
                        system_instruction=system_instruction
                    )

                    # Appel modèle (⚠️ pas de param system_instruction ici)
                    response = client.models.generate_content(
                        model=MODEL_NAME_SDK,
                        contents=contents,
                        config=req_config,
                    )

                    current_ocr_result = (response.text or '').strip() if response and hasattr(response, 'text') else ''
                    if not current_ocr_result:
                        current_ocr_result = 'ERROR: Empty response'

                    writer.writerow([current_ocr_result])

                    # Historique
                    ocr_result_n_minus_2 = ocr_result_n_minus_1
                    ocr_result_n_minus_1 = current_ocr_result

                except Exception as e_api:
                    print(f"\U0001F198 Error processing {chunk_filename}: {e_api}")
                    traceback.print_exc()
                    writer.writerow([f"ERROR: {type(e_api).__name__}"])
                finally:
                    time.sleep(1.0)  # petit délai

    except IOError as e:
        print(f"\U0001F198 CSV Error: {e}")
    except Exception as e_main:
        print(f"\U0001F198 Unexpected error: {e_main}")
        traceback.print_exc()

    print(f"\nCompleted processing (SDK) for: {os.path.basename(folder_path)}")
    print(f"CSV saved at: {output_csv_path}")

# Lancement
MAIN_FOLDER_PATH = '/content/drive/MyDrive/folder_partitioned_merged'
for subfolder in sorted(os.listdir(MAIN_FOLDER_PATH), key=natural_sort_key):
    full_path = os.path.join(MAIN_FOLDER_PATH, subfolder)
    if os.path.isdir(full_path):
        process_folder_sdk(full_path)



Starting OCR process (SDK) for folder: 'Avoir de la chance'
  Subject detected: 'Avoir de la chance'
  Output CSV will be: '/content/drive/MyDrive/folder_partitioned_merged/Avoir de la chance_sdk.csv'
  Found 38 chunks to process (SDK).


Processing Chunks (SDK):   0%|          | 0/38 [00:00<?, ?it/s]


Completed processing (SDK) for: Avoir de la chance
CSV saved at: /content/drive/MyDrive/folder_partitioned_merged/Avoir de la chance_sdk.csv

Starting OCR process (SDK) for folder: 'Avoir de la chance (1)'
  Subject detected: 'Avoir de la chance (1)'
  Output CSV will be: '/content/drive/MyDrive/folder_partitioned_merged/Avoir de la chance (1)_sdk.csv'
  Found 32 chunks to process (SDK).


Processing Chunks (SDK):   0%|          | 0/32 [00:00<?, ?it/s]


Completed processing (SDK) for: Avoir de la chance (1)
CSV saved at: /content/drive/MyDrive/folder_partitioned_merged/Avoir de la chance (1)_sdk.csv

Starting OCR process (SDK) for folder: 'L'idée de perfection'
  Subject detected: 'L'idée de perfection'
  Output CSV will be: '/content/drive/MyDrive/folder_partitioned_merged/L'idée de perfection_sdk.csv'
  Found 32 chunks to process (SDK).


...


Completed processing (SDK) for: L'idée de perfection
CSV saved at: /content/drive/MyDrive/folder_partitioned_merged/L'idée de perfection_sdk.csv

Starting OCR process (SDK) for folder: 'L'idée de perfection (1)'
  Subject detected: 'L'idée de perfection (1)'
  Output CSV will be: '/content/drive/MyDrive/folder_partitioned_merged/L'idée de perfection (1)_sdk.csv'
  Found 35 chunks to process (SDK).


Processing Chunks (SDK):   0%|          | 0/35 [00:00<?, ?it/s]


Completed processing (SDK) for: L'idée de perfection (1)
CSV saved at: /content/drive/MyDrive/folder_partitioned_merged/L'idée de perfection (1)_sdk.csv

Starting OCR process (SDK) for folder: 'Le savoir est-il désirable ?'
  Subject detected: 'Le savoir est-il désirable ?'
  Output CSV will be: '/content/drive/MyDrive/folder_partitioned_merged/Le savoir est-il désirable __sdk.csv'
  Found 34 chunks to process (SDK).


Processing Chunks (SDK):   0%|          | 0/34 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Example usage:



Starting SDK-based OCR for folder: 'Avoir de la chance'
  Output CSV will be: '/content/drive/MyDrive/folder_partitioned_merged/Avoir de la chance_sdk.csv'
  Found 38 chunks to process.
🛑 Error processing 00001.png: Part.from_bytes() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given


Traceback (most recent call last):
  File "/tmp/ipython-input-839473987.py", line 121, in process_folder_sdk
    contents.append(types.Part.from_bytes(image_bytes, mime_type='image/png'))
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Part.from_bytes() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given


🛑 Error processing 00002.png: Part.from_text() takes 1 positional argument but 2 were given


Traceback (most recent call last):
  File "/tmp/ipython-input-839473987.py", line 117, in process_folder_sdk
    contents = [types.Part.from_text(context_text)]
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Part.from_text() takes 1 positional argument but 2 were given


🛑 Error processing 00003.png: Part.from_text() takes 1 positional argument but 2 were given


Traceback (most recent call last):
  File "/tmp/ipython-input-839473987.py", line 117, in process_folder_sdk
    contents = [types.Part.from_text(context_text)]
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Part.from_text() takes 1 positional argument but 2 were given


KeyboardInterrupt: 